In [ ]:
# Static
import requests
from bs4 import BeautifulSoup

# Dinamic
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# BeautifulSoup

## Es 1

In [ ]:
headers = {'User-Agent':'Mozilla/5.0'}
url = 'http://www.scrapethissite.com/pages/simple/'
response = requests.get(url, headers=headers)

In [ ]:
status = response.status_code
html_doc = response.text

In [ ]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
titles = soup.find_all('h1')
titles[0].text.strip().split('\n')[0]

In [ ]:
meta_robots = soup.find('meta', attrs={'name':'robots'})

if meta_robots:
    if 'noindex' in meta_robots.get('content').lower():
        print('page no indexed')
else:
    print('page could be indexed')

In [ ]:
nations = []
for c in soup.find_all('div', class_='col-md-4 country'):
    country = {
        'name':c.h3.text.strip(),
        'capital':c.find('span',class_='country-capital').text,
        'population':c.find('span',class_='country-population').text,
        'area':c.find('span',class_='country-area').text
    }
    nations.append(country)

nations

In [ ]:
# some features utils

soup.head.title

# extract attribute
link = soup.find_all('a')[1]
link['href']

link.parent

soup.find('p').contents

# Selenium

In [ ]:
#options_f = webdriver.FirefoxOptions()
#driver_f = webdriver.Firefox(options=options_f)

options_c = webdriver.ChromeOptions()
driver_c = webdriver.Chrome(options=options_c)

#download:  'https://chromedriver.storage.googleapis.com/index.html?path=114.0.5735.90/'
#service_c = webdriver.ChromeService('path')
#driver_c = webdriver.Chrome(service=service_c)

In [ ]:
driver_c.get('https://books.toscrape.com/')

driver_c.title

driver_c.quit()

In [ ]:
# examples
element = WebDriverWait(driver_c, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'a'))
)

## Es 2

In [ ]:
options_c = webdriver.ChromeOptions()
driver_c = webdriver.Chrome(options=options_c)

In [ ]:
url = 'http://www.scrapethissite.com/pages/ajax-javascript/'

driver_c.get(url)

In [ ]:
WebDriverWait(driver_c, 10).until(
    EC.element_to_be_clickable((By.ID, '2015'))
).click()

In [ ]:
WebDriverWait(driver_c, 10).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'film'))
)

In [ ]:
films = driver_c.find_elements(By.CLASS_NAME, 'film')

In [ ]:
film_list = []
for f in films:
    film = {
        'title':f.find_element(By.CLASS_NAME,'film-title').text,
        'awards':f.find_element(By.CLASS_NAME,'film-awards').text
    }
    film_list.append(film)

film_list

## Es 3

In [ ]:
import requests
from bs4 import BeautifulSoup
import time # ritarda scraping tra una pagina e l'altra

In [ ]:
# options_c = webdriver.ChromeOptions()
# driver_c = webdriver.Chrome(options=options_c)

url = 'http://quotes.toscrape.com'
headers = {'User-Agent':'Mozilla/5.0'}
quotes = []

for i in range(1,6):
    urlx = f"{url}/page/{i}"
    response = requests.get(urlx, headers=headers)
    if(response.status_code==200):
        soup = BeautifulSoup(response.text, 'html.parser')
        quote_div = soup.find_all('div', class_='quote')
        
        for div in quote_div:
            q = {
                'author':div.find('small', class_='author').text,
                'quote':div.find('span', class_='text').text,
            }
            quotes.append(q)
    
        print(f"page {urlx} read")
        time.sleep(3)
    else:
        print(f'ERROR: Something wrong in page {i}. STATUS: {response.status_code}')
# quotes

In [ ]:
len(quotes)

### Save CSV

In [ ]:
import pandas as pd

df = pd.DataFrame(quotes, columns=['author','quote'])
df.to_csv('quotes.csv', index=False)

### Save JSON

In [ ]:
import json

with open('quotes.json','w', encoding='utf-8') as file:
    json.dump(quotes, file, ensure_ascii=False, indent=4)

### Save SQLite

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Create table
c.execute("CREATE TABLE QUOTES( author text, quote text )")

for q in quotes:
    c.execute(f"INSERT INTO QUOTES VALUES ({q['author']}, {q['quote']})")

conn.commit()
conn.close()